In [1]:
pip install rtree pygeos sodapy geopandas pyarrow

     |████████████████████████████████| 1.0 MB 22.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 68.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 99.8 MB/s eta 0:00:01
     |████████████████████████████████| 39.1 MB 99.2 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 3.1 MB/s  eta 0:00:01
     |████████████████████████████████| 6.3 MB 89.5 MB/s eta 0:00:01
     |████████████████████████████████| 15.6 MB 86.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 83.6 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 79.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 17.3 MB/s eta 0:00:01
  Attempting uninstall: requests
    Found existing installation: requests 2.24.0
    Uninstalling requests-2.24.0:
      Successfully uninstalled requests-2.24.0
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use

In [2]:
import rtree
import pygeos
import pandas as pd
from sodapy import Socrata
import geopandas as gpd
from shapely import wkt
from shapely.geometry import shape, MultiPolygon
import ast
import pyarrow.parquet as pq

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.3-CAPI-1.17.3) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
# Ran code below locally to create a new S3 bucket

# import boto3

# Initialize Boto Client for S3
# s3 = boto3.resource("s3", region_name="us-east-1")

# Create Bucket:
# s3.create_bucket(Bucket="elec-transit-y-jy")

In [4]:
def socrata_nyc_api_query(dataset_id):
  '''
  Assemble API query from NYC Open Data portal. Creates dataframe using dataset ID provided to function.

  Inputs:
    dataset_id (string): four by four string in the form XXXX-XXXX used to ID each dataset in the NYC Open Data portal.

  Returns:
    opendata_df (pandas dataframe): Dataframe of dataset from NYC Open Data portal.
  '''

  MyAppToken = "Wr8DB7EaAONFpZ4oE5v0ymJbT"
  client = Socrata("data.cityofnewyork.us", MyAppToken)

  # First 2000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy.
  results = client.get(dataset_id, limit=2000)

  # Convert to pandas DataFrame
  opendata_df = pd.DataFrame.from_records(results)

  return opendata_df

In [5]:
#Call above api query function using the dataset's unique identifier:
dataset_id = "755u-8jsi"
nyc_taxi_zone_df = socrata_nyc_api_query(dataset_id)
nyc_taxi_zone_df.head()

,objectid,shape_leng,the_geom,shape_area,zone,location_id,borough
0,1,0.116357453189,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.0007823067885,Newark Airport,1,EWR
1,2,0.43346966679,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.00486634037837,Jamaica Bay,2,Queens
2,3,0.0843411059012,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000314414156821,Allerton/Pelham Gardens,3,Bronx
3,4,0.0435665270921,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000111871946192,Alphabet City,4,Manhattan
4,5,0.0921464898574,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000497957489363,Arden Heights,5,Staten Island


In [6]:
# Inspect number of rows
len(nyc_taxi_zone_df)

263

In [7]:
# Inspect values in objectid column of taxi data
nyc_taxi_zone_df.objectid.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '24', '10', '11',
       '12', '13', '18', '25', '14', '15', '22', '23', '16', '17', '19',
       '20', '21', '26', '27', '28', '33', '29', '31', '32', '30', '34',
       '35', '36', '37', '38', '39', '40', '41', '45', '46', '42', '43',
       '44', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '72', '66',
       '67', '68', '73', '69', '70', '71', '79', '80', '74', '85', '86',
       '75', '76', '77', '78', '81', '87', '82', '83', '84', '88', '89',
       '90', '125', '91', '92', '93', '94', '95', '96', '97', '98', '99',
       '100', '101', '102', '103', '104', '105', '106', '107', '108',
       '109', '110', '111', '112', '113', '114', '115', '116', '117',
       '118', '119', '120', '149', '150', '121', '122', '123', '124',
       '126', '127', '128', '129', '134', '151', '130', '139', '131',
       '132', '133', '140', '135', '136', '137', '141', '142', '1

In [8]:
# Number of rows per taxi zone, confirming one row per taxi zone
grouped_by_zone_df = nyc_taxi_zone_df.groupby('objectid').size().reset_index(name='count')
group_by_zone_df = grouped_by_zone_df.sort_values(by='count', ascending=False)
grouped_by_zone_df

,objectid,count
0,1,1
1,10,1
2,100,1
3,101,1
4,102,1
...,...,...
258,95,1
259,96,1
260,97,1
261,98,1


In [9]:
# Inspect dataset
nyc_taxi_zone_df.head()

,objectid,shape_leng,the_geom,shape_area,zone,location_id,borough
0,1,0.116357453189,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.0007823067885,Newark Airport,1,EWR
1,2,0.43346966679,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.00486634037837,Jamaica Bay,2,Queens
2,3,0.0843411059012,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000314414156821,Allerton/Pelham Gardens,3,Bronx
3,4,0.0435665270921,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000111871946192,Alphabet City,4,Manhattan
4,5,0.0921464898574,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000497957489363,Arden Heights,5,Staten Island


In [10]:
def extract_coordinates(geom):
    '''
    Extract coordinates values from dictionary in NYC Taxi Zone dataset (755u-8jsi)
    and cast those coordinates as a MultiPolygon shape
    
    Input: 'geom' column of NYC Taxi Zone dataset
    
    Returns: 'geom' column as a MultiPolygon shape
    '''
    try:
        coordinates = geom['coordinates']
        return MultiPolygon(coordinates)
    except (ValueError, TypeError, KeyError):
        return None

def create_taxi_zone_geopandas_df(nyc_taxi_zone_df):
    '''
    Convert NYC Taxi Zone dataset to a geopandas dataframe
    
    Input: NYC Taxi Zone dataset from API query (755u-8jsi) as a pandas dataframe
    
    Return: NYC Taxi Zone dataset as a geopandas dataframe
    '''
    # Call extract_coordinates function in order to extract coordinates
    nyc_taxi_zone_df['geometry'] = nyc_taxi_zone_df['the_geom'].apply(extract_coordinates)
    
    # Create a GDF including all original columns and new geometry column
    taxi_zones_gdf = gpd.GeoDataFrame(nyc_taxi_zone_df, geometry='geometry')
    
    # Set the coordinate reference system (CRS) for the GeoDataFrame
    taxi_zones_gdf.set_crs(epsg=4326, inplace=True)
    
    return taxi_zones_gdf

In [11]:
# Call above functions in order to convert NYC Taxi Zone dataset to a geopandas dataframe
taxi_zones_gdf = create_taxi_zone_geopandas_df(nyc_taxi_zone_df)

In [12]:
# Read in New York EV data from s3 bucket
#ny_ev_parquet = spark.read.parquet('s3://final-project-nrel-stations-jy/station-parquet-files/NY_stations.parquet', header=True)

ny_ev_df = pd.read_parquet('s3://final-project-nrel-stations-jy/station-parquet-files/NY_stations.parquet')

In [13]:
# Inspect length of dataframe
len(ny_ev_df)

4754

In [14]:
# Inspect dataframe
ny_ev_df.head()

,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,maximum_vehicle_class,...,rd_blends_fr,rd_max_biodiesel_level,nps_unit_name,access_days_time_fr,intersection_directions_fr,bd_blends_fr,groups_with_access_code_fr,ev_pricing_fr,federal_agency,ev_network_ids
0,public,24 hours daily; call 866-809-4869 for Clean En...,CREDIT_CARD_ALWAYS,CleanEnergy D FuelMan M V Voyager Wright_Exp,2023-09-14,None,CNG,Public - Credit card at all times,108,HD,...,None,None,None,None,None,None,Public - Carte de crédit en tout temps,None,None,None
1,public,24 hours daily; call 866-809-4869 for Clean En...,CREDIT_CARD_ALWAYS,CleanEnergy D FleetOne FuelMan M Proprietor V ...,2024-01-09,None,CNG,Public - Credit card at all times,112,MD,...,None,None,None,None,None,None,Public - Carte de crédit en tout temps,None,None,None
2,public,8am-6pm M-F; call 718-204-4048 to arrange for ...,KEY_ALWAYS,None,2023-10-12,None,CNG,Public - Card key at all times,129,MD,...,None,None,None,None,None,None,Public - Carte-clé en tout temps,None,None,None
3,public,8am-8pm M-F; call 718-204-4048 to arrange for ...,KEY_ALWAYS,None,2023-10-12,None,CNG,Public - Card key at all times,130,MD,...,None,None,None,None,None,None,Public - Carte-clé en tout temps,None,None,None
4,public,"7am-11pm M-F, 7am-3pm Sat-Sun; call 718-204-40...",KEY_ALWAYS,None,2023-10-12,None,CNG,Public - Card key at all times,132,MD,...,None,None,None,None,None,None,Public - Carte-clé en tout temps,None,None,None


In [15]:
# Inspect columns in dataframe
ny_ev_df.columns

# Columns of interest = 'latitude' & 'longitude'

Index(['access_code', 'access_days_time', 'access_detail_code',
       'cards_accepted', 'date_last_confirmed', 'expected_date',
       'fuel_type_code', 'groups_with_access_code', 'id',
       'maximum_vehicle_class', 'open_date', 'owner_type_code',
       'restricted_access', 'status_code', 'facility_type', 'station_name',
       'station_phone', 'updated_at', 'geocode_status', 'latitude',
       'longitude', 'city', 'country', 'intersection_directions', 'plus4',
       'state', 'street_address', 'zip', 'bd_blends', 'cng_dispenser_num',
       'cng_fill_type_code', 'cng_has_rng', 'cng_psi', 'cng_renewable_source',
       'cng_total_compression', 'cng_total_storage', 'cng_vehicle_class',
       'e85_blender_pump', 'e85_other_ethanol_blends', 'ev_connector_types',
       'ev_dc_fast_num', 'ev_level1_evse_num', 'ev_level2_evse_num',
       'ev_network', 'ev_network_web', 'ev_other_evse', 'ev_pricing',
       'ev_renewable_source', 'ev_workplace_charging', 'hy_is_retail',
       'hy_pres

In [16]:
def create_ev_chargers_geopandas_df(ny_ev_df):
    '''
    Convert EV charger dataset to a geopandas dataframe
    
    Input: EV charger dataset as a pandas dataframe
    
    Return: EV charger dataset as a geopandas dataframe
    '''
    # Converting NY ev chargers dataset to gpd
    ny_ev_gdf = gpd.GeoDataFrame(ny_ev_df, geometry=gpd.points_from_xy(ny_ev_df.longitude, ny_ev_df.latitude))
    
    # Set the coordinate reference system (CRS) for the GeoDataFrame
    ny_ev_gdf.set_crs(epsg=4326, inplace=True)
    
    return ny_ev_gdf

In [18]:
# Inspect ny_ev_gdf
ny_ev_gdf = create_ev_chargers_geopandas_df(ny_ev_df)
ny_ev_gdf.head()

,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,maximum_vehicle_class,...,rd_max_biodiesel_level,nps_unit_name,access_days_time_fr,intersection_directions_fr,bd_blends_fr,groups_with_access_code_fr,ev_pricing_fr,federal_agency,ev_network_ids,geometry
0,public,24 hours daily; call 866-809-4869 for Clean En...,CREDIT_CARD_ALWAYS,CleanEnergy D FuelMan M V Voyager Wright_Exp,2023-09-14,None,CNG,Public - Credit card at all times,108,HD,...,None,None,None,None,None,Public - Carte de crédit en tout temps,None,None,None,POINT (-73.93231 40.71804)
1,public,24 hours daily; call 866-809-4869 for Clean En...,CREDIT_CARD_ALWAYS,CleanEnergy D FleetOne FuelMan M Proprietor V ...,2024-01-09,None,CNG,Public - Credit card at all times,112,MD,...,None,None,None,None,None,Public - Carte de crédit en tout temps,None,None,None,POINT (-73.91834 40.64554)
2,public,8am-6pm M-F; call 718-204-4048 to arrange for ...,KEY_ALWAYS,None,2023-10-12,None,CNG,Public - Card key at all times,129,MD,...,None,None,None,None,None,Public - Carte-clé en tout temps,None,None,None,POINT (-73.86157 40.84419)
3,public,8am-8pm M-F; call 718-204-4048 to arrange for ...,KEY_ALWAYS,None,2023-10-12,None,CNG,Public - Card key at all times,130,MD,...,None,None,None,None,None,Public - Carte-clé en tout temps,None,None,None,POINT (-73.69305 40.97900)
4,public,"7am-11pm M-F, 7am-3pm Sat-Sun; call 718-204-40...",KEY_ALWAYS,None,2023-10-12,None,CNG,Public - Card key at all times,132,MD,...,None,None,None,None,None,Public - Carte-clé en tout temps,None,None,None,POINT (-73.84307 40.76962)


In [19]:
# Inspect length of geodataframe
len(ny_ev_gdf)

4754

In [23]:
def merge_ev_charger_and_taxi_zones(ny_ev_gdf, taxi_zones_gdf):
    '''
    Merge together electric vehicle charger geopandas dataframe with 
    taxi zones geopandas dataframe
    
    Inputs:
    - ny_ev_gdf (geopandas dataframe)
    - taxi_zones_gdf (geopandas dataframe)
    
    Returns: ev_chargers_in_taxi_zones_gpd (geopandas dataframe)
    '''
    
    # Do an inner spatial join of taxi zones and ev chargers to determine which ev chargers are in which taxi zones
    ev_chargers_in_taxi_zones_gpd = gpd.sjoin(ny_ev_gdf, taxi_zones_gdf, how='inner', op='within', rsuffix = 'taxi_zone')
    
    # Re-add geometry of taxi zone back into dataset
    ev_chargers_in_taxi_zones_gpd['geometry_taxi_zone'] = ev_chargers_in_taxi_zones_gpd['the_geom'].apply(extract_coordinates)
    ev_chargers_in_taxi_zones_gpd = gpd.GeoDataFrame(ev_chargers_in_taxi_zones_gpd, geometry='geometry_taxi_zone')
    
    return ev_chargers_in_taxi_zones_gpd

In [24]:
# Call function to do spatial merge with ev charging stations and taxi zones
ev_chargers_in_taxi_zones_gpd = merge_ev_charger_and_taxi_zones(ny_ev_gdf, taxi_zones_gdf)

# Inspect length of geodataframe
len(ev_chargers_in_taxi_zones_gpd)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


798

In [25]:
# Inspect head of merged geodataframe
ev_chargers_in_taxi_zones_gpd.head()

,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,maximum_vehicle_class,...,geometry,index_taxi_zone,objectid,shape_leng,the_geom,shape_area,zone,location_id,borough,geometry_taxi_zone
0,public,24 hours daily; call 866-809-4869 for Clean En...,CREDIT_CARD_ALWAYS,CleanEnergy D FuelMan M V Voyager Wright_Exp,2023-09-14,None,CNG,Public - Credit card at all times,108,HD,...,POINT (-73.93231 40.71804),74,80,0.117212621448,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.00040732245622,East Williamsburg,80,Brooklyn,"MULTIPOLYGON (((-73.93175 40.72805, -73.93168 ..."
170,private,None,None,None,2024-01-09,None,BD,Private,52974,HD,...,POINT (-73.93151 40.71805),74,80,0.117212621448,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.00040732245622,East Williamsburg,80,Brooklyn,"MULTIPOLYGON (((-73.93175 40.72805, -73.93168 ..."
1629,private,None,None,None,2024-05-09,None,ELEC,Private,167695,MD,...,POINT (-73.93151 40.71805),74,80,0.117212621448,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.00040732245622,East Williamsburg,80,Brooklyn,"MULTIPOLYGON (((-73.93175 40.72805, -73.93168 ..."
2598,public,24 hours daily,None,None,2024-05-22,None,ELEC,Public,199108,None,...,POINT (-73.95319 40.71432),74,80,0.117212621448,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.00040732245622,East Williamsburg,80,Brooklyn,"MULTIPOLYGON (((-73.93175 40.72805, -73.93168 ..."
3036,public,24 hours daily; Also accepts payment via the C...,None,A CREDIT D Debit M V Voyager Wright_Exp,2024-03-14,None,ELEC,Public,223588,LD,...,POINT (-73.93310 40.71360),74,80,0.117212621448,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.00040732245622,East Williamsburg,80,Brooklyn,"MULTIPOLYGON (((-73.93175 40.72805, -73.93168 ..."


In [26]:
# Inspect head of merged geodataframe
ev_chargers_in_taxi_zones_gpd.columns

Index(['access_code', 'access_days_time', 'access_detail_code',
       'cards_accepted', 'date_last_confirmed', 'expected_date',
       'fuel_type_code', 'groups_with_access_code', 'id',
       'maximum_vehicle_class', 'open_date', 'owner_type_code',
       'restricted_access', 'status_code', 'facility_type', 'station_name',
       'station_phone', 'updated_at', 'geocode_status', 'latitude',
       'longitude', 'city', 'country', 'intersection_directions', 'plus4',
       'state', 'street_address', 'zip', 'bd_blends', 'cng_dispenser_num',
       'cng_fill_type_code', 'cng_has_rng', 'cng_psi', 'cng_renewable_source',
       'cng_total_compression', 'cng_total_storage', 'cng_vehicle_class',
       'e85_blender_pump', 'e85_other_ethanol_blends', 'ev_connector_types',
       'ev_dc_fast_num', 'ev_level1_evse_num', 'ev_level2_evse_num',
       'ev_network', 'ev_network_web', 'ev_other_evse', 'ev_pricing',
       'ev_renewable_source', 'ev_workplace_charging', 'hy_is_retail',
       'hy_pres

In [34]:
# Number of rows per taxi zone, confirming one row per taxi zone
grouped_by_zone_and_ev_count_df = ev_chargers_in_taxi_zones_gpd.groupby('objectid').size().reset_index(name='count')
grouped_by_zone_and_ev_count_df = grouped_by_zone_and_ev_count_df.sort_values(by='count', ascending=False)
grouped_by_zone_and_ev_count_df.rename(columns={'count': 'ev_count'}, inplace=True)
grouped_by_zone_and_ev_count_df.rename(columns={'objectid': 'taxi_zone_id'}, inplace=True)
grouped_by_zone_and_ev_count_df = grouped_by_zone_and_ev_count_df.sort_values(by='taxi_zone_id')
grouped_by_zone_and_ev_count_df.head(25)

# 158 of the 262 taxi zones have an electric vehicle charger

,taxi_zone_id,ev_count
0,101,1
1,102,1
2,106,1
3,107,10
4,112,2
5,113,11
6,114,4
7,117,1
8,118,4
9,125,1


In [37]:
# Read in NYC taxi rides data, aggregated by pickup taxi zone
aggByPUZone_nyc_taxi_data = pd.read_parquet('s3://elec-transit-y-jy/nyc_taxi_rides_2019_aggByPU.parquet/nyc_taxi_rides_2019_aggByPU.parquet')
aggByPUZone_nyc_taxi_data.rename(columns={'PULocationID': 'taxi_zone_id'}, inplace=True)
aggByPUZone_nyc_taxi_data.head(10)

,taxi_zone_id,PU_avg_passenger_count,PU_avg_distance,PU_avg_ratecodeid,PU_avg_fare_amount,PUavg_total_amount,PU_trip_counts
0,1,1.509902,1.586234,4.696120,80.797817,96.730146,8644
1,10,1.545722,13.437876,2.184312,48.449535,62.339703,31129
2,100,1.546737,2.358822,1.036906,11.781948,17.193274,1725187
3,101,1.186228,8.505512,1.961078,34.430577,37.710148,2237
4,102,1.365021,5.369619,1.577697,25.272136,28.768841,2097
5,104,1.000000,8.180000,1.000000,25.000000,31.560000,1
6,105,1.538462,4.355897,1.059829,16.322821,21.977778,117
7,106,1.468310,3.419813,1.132896,14.958941,18.627953,9811
8,107,1.555881,2.172276,1.024819,10.838164,16.358919,1896669
9,108,1.305556,6.625020,1.715812,31.985622,35.803425,1711


In [40]:
# Read in NYC taxi rides data, aggregated by dropoff taxi zone
aggByDOZone_nyc_taxi_data = pd.read_parquet('s3://elec-transit-y-jy/nyc_taxi_rides_2019_aggByDO.parquet/nyc_taxi_rides_2019_aggByDO.parquet')
aggByDOZone_nyc_taxi_data.rename(columns={'DOLocationID': 'taxi_zone_id'}, inplace=True)
aggByDOZone_nyc_taxi_data.head(10)

,taxi_zone_id,DO_avg_passenger_count,DO_avg_distance,DO_avg_ratecodeid,DO_avg_fare_amount,DO_avg_total_amount,DO_trip_counts
0,1,1.637576,17.360809,3.156715,71.860149,98.749078,175514
1,10,1.525943,5.289383,1.148105,18.664750,21.696166,64602
2,100,1.585192,2.153654,1.042480,11.796396,17.356973,1335532
3,101,1.481631,12.604188,1.394667,39.259592,44.606791,7011
4,102,1.538932,8.544134,1.089423,29.020189,34.431938,12650
5,105,1.540230,3.464598,1.149425,15.144253,20.588621,87
6,106,1.544363,7.034566,1.033491,24.746354,32.078894,38850
7,107,1.550974,2.045252,1.028209,10.172589,15.753829,1735914
8,108,1.531621,13.540840,1.221523,41.983334,47.716095,6320
9,109,1.538539,22.639713,1.175175,66.267144,90.058271,1012


In [42]:
len(nyc_taxi_rides_INTERMEDIATE)


262

In [56]:
def merge_nyc_taxi_rides_zones_ev_chargers(grouped_by_zone_and_ev_count_df, aggByPUZone_nyc_taxi_data, aggByDOZone_nyc_taxi_data):
    '''
    Merge together dataset containing EV chargers and taxi zones with NYC taxi ride data from 2019
    
    Inputs: 
     - grouped_by_zone_and_ev_count_df: Merged EV charger & taxi zone, aggregated by taxi zone
     - aggByPUZone_nyc_taxi_data: NYC taxi ride data aggregated by pickup zone
     - aggByDOZone_nyc_taxi_data: NYC taxi ride data aggregated by dropoff zone
    
    Return: NYC Taxi Zone dataset as a geopandas dataframe
    '''
    
    # Step 1 of merge: combining pickups and dropoffs
    nyc_taxi_rides_INTERMEDIATE = aggByPUZone_nyc_taxi_data.merge(aggByDOZone_nyc_taxi_data, on='taxi_zone_id', how='inner')
    
    # Step 2 of merge: cominings rides and ev charger counts
    nyc_taxi_rides_zones_evChargers_INTERMEDIATE = nyc_taxi_rides_INTERMEDIATE.merge(grouped_by_zone_and_ev_count_df, on='taxi_zone_id', how='left')
    nyc_taxi_rides_zones_evChargers_INTERMEDIATE['ev_count'].fillna(0.0, inplace=True)
    
    # Add taxi zone shapes back in 
    taxi_zones_gdf = create_taxi_zone_geopandas_df(nyc_taxi_zone_df)
    taxi_zones_gdf.rename(columns={'objectid': 'taxi_zone_id'}, inplace=True)
    
    # Step 3 of merge: adding in shape of taxi zone
    nyc_taxi_rides_zones_evChargers = nyc_taxi_rides_zones_evChargers_INTERMEDIATE.merge(taxi_zones_gdf, on='taxi_zone_id', how='left')
    
    return nyc_taxi_rides_zones_evChargers

In [57]:
nyc_taxi_rides_zones_evChargers = merge_nyc_taxi_rides_zones_ev_chargers(grouped_by_zone_and_ev_count_df, 
                                                                         aggByPUZone_nyc_taxi_data, 
                                                                         aggByDOZone_nyc_taxi_data)
nyc_taxi_rides_zones_evChargers.head(10)

,taxi_zone_id,PU_avg_passenger_count,PU_avg_distance,PU_avg_ratecodeid,PU_avg_fare_amount,PUavg_total_amount,PU_trip_counts,DO_avg_passenger_count,DO_avg_distance,DO_avg_ratecodeid,...,DO_avg_total_amount,DO_trip_counts,ev_count,shape_leng,the_geom,shape_area,zone,location_id,borough,geometry
0,1,1.509902,1.586234,4.696120,80.797817,96.730146,8644,1.637576,17.360809,3.156715,...,98.749078,175514,0.0,0.116357453189,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.0007823067885,Newark Airport,1,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,10,1.545722,13.437876,2.184312,48.449535,62.339703,31129,1.525943,5.289383,1.148105,...,21.696166,64602,0.0,0.0998394794152,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000435823818081,Baisley Park,10,Queens,"MULTIPOLYGON (((-73.78327 40.68999, -73.78234 ..."
2,100,1.546737,2.358822,1.036906,11.781948,17.193274,1725187,1.585192,2.153654,1.042480,...,17.356973,1335532,0.0,0.0248131090342,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.0000374700210291,Garment District,100,Manhattan,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ..."
3,101,1.186228,8.505512,1.961078,34.430577,37.710148,2237,1.481631,12.604188,1.394667,...,44.606791,7011,1.0,0.101709836277,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000452342528877,Glen Oaks,101,Queens,"MULTIPOLYGON (((-73.70135 40.75078, -73.70117 ..."
4,102,1.365021,5.369619,1.577697,25.272136,28.768841,2097,1.538932,8.544134,1.089423,...,34.431938,12650,1.0,0.136900484646,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000296595466345,Glendale,102,Queens,"MULTIPOLYGON (((-73.85596 40.70643, -73.85594 ..."
5,105,1.538462,4.355897,1.059829,16.322821,21.977778,117,1.540230,3.464598,1.149425,...,20.588621,87,0.0,0.0774253398314,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000368636392805,Governor's Island/Ellis Island/Liberty Island,103,Manhattan,"MULTIPOLYGON (((-74.01675 40.69334, -74.01540 ..."
6,106,1.468310,3.419813,1.132896,14.958941,18.627953,9811,1.544363,7.034566,1.033491,...,32.078894,38850,1.0,0.0492110119798,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000113616826567,Gowanus,106,Brooklyn,"MULTIPOLYGON (((-73.98610 40.68101, -73.98414 ..."
7,107,1.555881,2.172276,1.024819,10.838164,16.358919,1896669,1.550974,2.045252,1.028209,...,15.753829,1735914,10.0,0.0380413645908,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.0000746112192675,Gramercy,107,Manhattan,"MULTIPOLYGON (((-73.98239 40.73141, -73.98256 ..."
8,108,1.305556,6.625020,1.715812,31.985622,35.803425,1711,1.531621,13.540840,1.221523,...,47.716095,6320,0.0,0.11117141227,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.000352866009465,Gravesend,108,Brooklyn,"MULTIPOLYGON (((-73.97148 40.59717, -73.97133 ..."
9,109,1.402778,8.661233,1.986111,47.029589,52.621096,73,1.538539,22.639713,1.175175,...,90.058271,1012,0.0,0.178267819599,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.00116960076185,Great Kills,109,Staten Island,"MULTIPOLYGON (((-74.14706 40.57051, -74.14699 ..."


In [58]:
# Check dataframe includes all taxi zones
len(nyc_taxi_rides_zones_evChargers)

262

In [60]:
# Write merged data to S3 bucket of choice as csv
nyc_taxi_rides_zones_evChargers.to_csv("s3://elec-transit-y-jy/nyc_taxi_rides_zones_evChargers.csv")

In [63]:
# Write merged data to S3 bucket of choice as parquet
nyc_taxi_rides_zones_evChargers['geometry_wkt'] = nyc_taxi_rides_zones_evChargers['geometry'].apply(lambda x: x.wkt if x is not None else None)
nyc_taxi_merged_wkt = nyc_taxi_rides_zones_evChargers.drop(columns=['geometry'])
nyc_taxi_merged_wkt.to_parquet("s3://elec-transit-y-jy/nyc_taxi_rides_zones_evChargers.parquet")